In [1]:
# 2024.06.26 어제꺼
import os
import subprocess
from pyarrow import fs
import pyarrow as pa

classpath = subprocess.Popen(["/home/hadoop/hadoop/bin/hdfs", "classpath", "--glob"], stdout=subprocess.PIPE).communicate()[0]
os.environ["CLASSPATH"] = classpath.decode("utf-8")
hdfs = fs.HadoopFileSystem(host='ip', port=port, user='hadoop')

2024-06-27 00:01:55,796 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [25]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("bsh").getOrCreate()

spark

24/06/27 01:14:17 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
dir_path = '/'

# 디렉토리 목록 가져오기
file_infos = hdfs.get_file_info(pa.fs.FileSelector(dir_path, recursive=False))  
for file_info in file_infos:
    # print(file_info.path)
    pass

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("20240626").getOrCreate()

df = spark.read.csv("hdfs:////encore/tpss_bcycl_od_statnhm_202001.csv", encoding='cp949', header=True)


data = [("James", "Sales", 3000), \
    ("Michael", "Sales", 4600), \
    ("Robert", "Sales", 4100), \
    ("Maria", "Finance", 3000), \
    ("James", "Sales", 3000), \
    ("Scott", "Finance", 3300), \
    ("Jen", "Finance", 3900), \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000), \
    ("Saif", "Sales", 4100) \
  ]

24/06/27 00:01:58 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
columns= ["employee_name", "department", "salary"]
test = spark.createDataFrame(data = data, schema = columns)

df.select("시작_대여소ID").distinct().count()

1543

In [8]:
from pyspark.sql.functions import col,mean

In [9]:
df2 = df.withColumn("전체이용거리(m)", col("전체이용거리(m)").cast('int'))
from pyspark.sql.types import IntegerType
df3 = df.withColumn("전체이용거리(m)", col("전체이용거리(m)").cast(IntegerType()))
df2.withColumn("전체이용시간(분)", col("전체이용시간(분)").cast('int')).select(mean(col("전체이용시간(분)"))).show()


+---------------------+
|avg(전체이용시간(분))|
+---------------------+
|   19.249510036830372|
+---------------------+



In [13]:
from pyspark.sql.types import StructType, IntegerType, StringType

In [15]:
schema = StructType().\
      add('기준_날짜',  StringType(), True ).\
        add('구분코드',  StringType(), True ).\
        add('기준_시간',  StringType(), True ).\
        add('시작_대여소ID',  StringType(), True ).\
        add('시작_대여소명',  StringType(), True ).\
        add('종료_대여소ID',  StringType(), True ).\
        add('종료_대여소명',  StringType(), True ).\
        add('전체건수',  IntegerType(), True ).\
        add('전체이용시간(분)',  IntegerType(), True ).\
        add('전체이용거리(m)' , IntegerType(), True )

tmp = spark.read.csv("hdfs:////encore/tpss_bcycl_od_statnhm_202001.csv", encoding='cp949', header=True, schema=schema)

from pyspark.sql.functions import mean, col, max, min, stddev

tmp.select( stddev('전체이용시간(분)')).show()


tmp.select( mean('전체이용시간(분)')).show()


+-----------------------------+
|stddev_samp(전체이용시간(분))|
+-----------------------------+
|            26.96801174397804|
+-----------------------------+



+---------------------+
|avg(전체이용시간(분))|
+---------------------+
|   19.249510036830372|
+---------------------+



In [16]:
# !hdfs dfs -get /encore/tpss_bcycl_od_statnhm_202001.csv ./
# !hdfs dfs -get /encore/tpss_bcycl_od_statnhm_202007.csv ./

In [17]:
from pyspark.sql.functions import coalesce
tmp.select(coalesce('전체이용시간(분)')).count()


1669003

In [18]:
# Returns the first column that is not null.


cDf = spark.createDataFrame([(None, None), (1, None), (None, 2)], ("a", "b"))
cDf.show()




cDf = spark.createDataFrame([(None, None, None), (None, 1, None), (None, 2, 6), (6, 3, 4)], ("a", "b", "c"))


+----+----+
|   a|   b|
+----+----+
|null|null|
|   1|null|
|null|   2|
+----+----+



In [19]:
cDf.show()

+----+----+----+
|   a|   b|   c|
+----+----+----+
|null|null|null|
|null|   1|null|
|null|   2|   6|
|   6|   3|   4|
+----+----+----+



In [21]:
from pyspark.sql.functions import isnull

In [22]:
cDf.select(isnull("a")).show()


cDf.filter(isnull("a") | isnull("b")).count()



from pyspark.sql.functions import col, when, count
cDf.agg(*[count(when(isnull(x), x)).\
          alias("null") 
             for x in cDf.columns]).show()

when_test = spark.createDataFrame([("A", 10), ("B", 20), ("C", 30)], ["category", "value"])

df_new = when_test.withColumn("new_col", when(col("value") > 20, "high").otherwise("low"))
df_new.show()



+-----------+
|(a IS NULL)|
+-----------+
|       true|
|       true|
|       true|
|      false|
+-----------+

+----+----+----+
|null|null|null|
+----+----+----+
|   3|   1|   2|
+----+----+----+

+--------+-----+-------+
|category|value|new_col|
+--------+-----+-------+
|       A|   10|    low|
|       B|   20|    low|
|       C|   30|   high|
+--------+-----+-------+



In [23]:
tmp.agg(*[count(when(isnull(x), x)).\
          alias(x) 
             for x in tmp.columns]).show()


+---------+--------+---------+-------------+-------------+-------------+-------------+--------+----------------+---------------+
|기준_날짜|구분코드|기준_시간|시작_대여소ID|시작_대여소명|종료_대여소ID|종료_대여소명|전체건수|전체이용시간(분)|전체이용거리(m)|
+---------+--------+---------+-------------+-------------+-------------+-------------+--------+----------------+---------------+
|        0|       0|        0|            0|            0|            0|            0|       0|               0|              0|
+---------+--------+---------+-------------+-------------+-------------+-------------+--------+----------------+---------------+



In [24]:
!hdfs dfs -get /encore/tpss_bcycl_od_statnhm_202001.csv ./
!hdfs dfs -get /encore/tpss_bcycl_od_statnhm_202007.csv ./

get: `./tpss_bcycl_od_statnhm_202001.csv': File exists
